In [37]:
using DynamicPolynomials, CPUTime, LinearAlgebra, LightGraphs, JuMP, MosekTools, RowEchelon


start = time()





# define polnomial variables
@polyvar b d f B D F
x=[b;d;f;B;D;F]; n=length(x)

#inequalities polynomial
g = [] ; m=length(g)

#inequalities polynomial
h = [ (b-d)*(B-D) - 2*F + 2;
 (b-d)*(B+D-2*F) + 2*(B-D);
 (b-d)*(b-d) - 2*(b+d) + f + 1;
 B^2*b^3 - 1;
 D^2*d^3 - 1;
 F^2*f^3 - 1] ; l=length(h)



# small parameter for L
eps = 1e-2

k=2 # relaxed order

k_sparse=2 # sparse order

# for rank of moment matrix
TOL=1e-4

# parameter for pivot of rref
tau=1e-3

# Define centers and square of radius
a0=zeros(Float64,(n, 1)); a = Matrix{Float64}(I, n, n)

6×6 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [38]:
theta=1+x'*x

println("Determine L0:")
 
println("---------------------------------------------------------------------------------------")

# Polynomial to optimize 
f = [x-a0]'*[x-a0]

d=1

if m==0
    g=[f]
else
    g=[g;f]
end
m=length(g)

# Degree of inequalities polynomials
dg = []
for i = 1:m
    dg = [dg; ceil(Int,DynamicPolynomials.degree(leadingmonomial(g[i]))/2)]
end

dh = []
for i = 1:l
    dh = [dh; ceil(Int,DynamicPolynomials.degree(leadingmonomial(h[i]))/2)]
end



rf=binomial(k+d+n,n)

rg=[];
for i = 1:m
    rg=[rg;binomial(k+d-dg[i]+n,n)]
end

rh=[];
for i = 1:l
    rh=[rh;binomial(k+d-dh[i]+n,n)]
end


v=monomials(x, 0)
for j=1:k+d
    v=[v;monomials(x, j)]
end

Uf=union(v.^2,monomials(theta^k*(f+1/eps+eps*theta^d)))

Ug=[]
for i = 1:m
    Uf=union(Uf,monomials(g[i]))
    push!(Ug,[])
end

Uh=[]
for i = 1:l
    Uf=union(Uf,monomials(h[i]))
    push!(Uh,[])
end


Tf=v*v'
Tg=[]
for i=1:m
    push!(Tg,g[i].*v[1:rg[i]]*v[1:rg[i]]')
end

Th=[]
for i=1:l
    push!(Th,h[i].*v[1:rh[i]]*v[1:rh[i]]')
end


length_ind_block_f=[]
ind_block_f=[]

length_ind_block_g=[]
ind_block_g=[]

for i=1:m
    push!(length_ind_block_g,[])
    push!(ind_block_g,[])
end

length_ind_block_h=[]
ind_block_h=[]

for i=1:l
    push!(length_ind_block_h,[])
    push!(ind_block_h,[])
end


for t=0:k_sparse-1
    U=Uf

    for i=1:m
        if length(Ug[i])!=0
            U=union(U,Ug[i])
        end
    end
    for i=1:l
        if length(Uh[i])!=0
            U=union(U,Uh[i])
        end
    end
    
    
    Cf=zeros(rf,rf)
    
    Cf[findall(p -> p in U, Tf)].=1
    
    Cf[diagind(Cf)] .= 0
    
    Cg=[]
    for i=1:m
        C=zeros(rg[i],rg[i])
        
        for p=1:rg[i]
            for q=1:rg[i]
                if length(intersect(monomials(Tg[i][p,q]),U))!=0
                    C[p,q]=1
                end
            end
        end
        C[diagind(C)] .= 0
        push!(Cg,C)
    end
        
    Ch=[]
    for i=1:l
        C=zeros(rh[i],rh[i])

        for p=1:rh[i]
            for q=1:rh[i]
                if length(intersect(monomials(Th[i][p,q]),U))!=0
                    C[p,q]=1
                end
            end
        end
        C[diagind(C)] .= 0
        push!(Ch,C)
    end
    
    #maximal cliques

    grap = SimpleGraph(Cf)
    
    ind_block_f=connected_components(grap)
    
    length_ind_block_f=length(ind_block_f)
    
    for j=1:length_ind_block_f
        Uf=[Uf;vec(v[ind_block_f[j]]*v[ind_block_f[j]]')]  
    end
    Uf=unique(Uf)
    
    for i=1:m
        
        grap = SimpleGraph(Cg[i])

        ind_block_g[i]=connected_components(grap)

        length_ind_block_g[i]=length(ind_block_g[i])

        for j=1:length_ind_block_g[i]
            mono=monomials(g[i])
            Ug[i]=vec(mono[1]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]')
            for p=2:length(mono)
                Ug[i]=union(Ug[i],vec(mono[p]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]'))
            end
        end
    end
    
    

    
    for i=1:l
        
        grap = SimpleGraph(Ch[i])

        ind_block_h[i]=connected_components(grap)

        length_ind_block_h[i]=length(ind_block_h[i])

        for j=1:length_ind_block_h[i]
            mono=monomials(h[i])
            Uh[i]=vec(mono[1]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]')
            for p=2:length(mono)
                Uh[i]=union(Uh[i],vec(mono[p]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]'))
            end
        end
    end    
  
end

elapsed = time() - start
println("elapsed time = ",elapsed)

Determine L0:
---------------------------------------------------------------------------------------
elapsed time = 26.296000003814697


In [39]:
ind_block_f

1-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  75, 76, 77, 78, 79, 80, 81, 82, 83, 84]

In [40]:
ind_block_g

1-element Array{Any,1}:
 Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]

In [41]:
ind_block_h

6-element Array{Any,1}:
 Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]
 Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]
 Array{Int64,1}[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  19, 20, 21, 22, 23, 24, 25, 26, 27, 28]]
 Array{Int64,1}[[1]]                                                                       
 Array{Int64,1}[[1]]                                                                       
 Array{Int64,1}[[1]]                                                                       

In [42]:
model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))

wSOS=0

G=[]
for i=1:length_ind_block_f
    push!(G, @variable(model, [1:length(ind_block_f[i]), 1:length(ind_block_f[i])],PSD)) 
end

for i=1:length_ind_block_f
    wSOS = wSOS + v[ind_block_f[i]]'*G[i]*v[ind_block_f[i]]   
end


Gg=[]
for j=1:m
    push!(Gg,[])
    for i=1:length_ind_block_g[j]
        push!(Gg[j], @variable(model, [1:length(ind_block_g[j][i]), 1:length(ind_block_g[j][i])],PSD))
    end
end

for j=1:m
    for i=1:length_ind_block_g[j]
        wSOS = wSOS + g[j]*v[ind_block_g[j][i]]'*Gg[j][i]*v[ind_block_g[j][i]]   
    end
end


Gh=[]
length_Gh=[]
vh=[]
for j=1:l
    push!(Gh,[])
    push!(length_Gh,[])
    push!(vh,[])
    for i=1:length_ind_block_h[j]
        push!(vh[j], unique(vec(v[ind_block_h[j][i]]*v[ind_block_h[j][i]]')))
        push!(length_Gh[j], length(vh[j][i]))
        push!(Gh[j], @variable(model, [1:length_Gh[j][i]]))
    end
end

for j=1:l
    for i=1:length_ind_block_h[j]
        wSOS = wSOS + h[j]*vh[j][i]'*Gh[j][i]   
    end
end



@variable(model, lambda)

@constraint(model, coefficients(theta^k*(f-lambda+eps*theta^d) - wSOS).== 0)

@objective(model, Max, lambda)

optimize!(model)

L0 = value(lambda)

status= termination_status(model)

println("termination status = ", status)

println("L0 = ", L0)

elapsed = time() - start
println("elapsed time = ",elapsed)

println("=======================================================================================")


termination status = OPTIMAL
L0 = 4.838694626682971
elapsed time = 29.722000122070312


In [ ]:
# Define omegat, t=0,...,n
omega0 = 0; omega = zeros(n)

println("Determine omega",0,":")

println("---------------------------------------------------------------------------------------")

#inequalities polynomial
g[end]=L0-f

# Degree of inequalities polynomials
dg = []
for i = 1:m
    dg = [dg; ceil(Int,DynamicPolynomials.degree(leadingmonomial(g[i]))/2)]
end

dh = []
for i = 1:l
    dh = [dh; ceil(Int,DynamicPolynomials.degree(leadingmonomial(h[i]))/2)]
end

d=maximum([dg;dh;1])

rf=binomial(k+d+n,n)

rg=[];
for i = 1:m
    rg=[rg;binomial(k+d-dg[i]+n,n)]
end

rh=[];
for i = 1:l
    rh=[rh;binomial(k+d-dh[i]+n,n)]
end


v=monomials(x, 0)
for j=1:k+d
    v=[v;monomials(x, j)]
end



Uf=union(v.^2,monomials(f))

Ug=[]
for i = 1:m
    Uf=union(Uf,monomials(g[i]))
    push!(Ug,[])
end

Uh=[]
for i = 1:l
    Uf=union(Uf,monomials(h[i]))
    push!(Uh,[])
end


Tf=v*v'
Tg=[]
for i=1:m
    push!(Tg,g[i].*v[1:rg[i]]*v[1:rg[i]]')
end

Th=[]
for i=1:l
    push!(Th,h[i].*v[1:rh[i]]*v[1:rh[i]]')
end



length_ind_block_f=[]
ind_block_f=[]

length_ind_block_g=[]
ind_block_g=[]

for i=1:m
    push!(length_ind_block_g,[])
    push!(ind_block_g,[])
end

length_ind_block_h=[]
ind_block_h=[]

for i=1:l
    push!(length_ind_block_h,[])
    push!(ind_block_h,[])
end


for t=0:k_sparse-1
    U=Uf
    for i=1:m
        if length(Ug[i])!=0
            U=union(U,Ug[i])
        end
    end
    for i=1:l
        if length(Uh[i])!=0
            U=union(U,Uh[i])
        end
    end
    
    Cf=zeros(rf,rf)
    
    Cf[findall(p -> p in U, Tf)].=1
    
    Cf[diagind(Cf)] .= 0
    
    Cg=[]
    for i=1:m
        C=zeros(rg[i],rg[i])
        
        for p=1:rg[i]
            for q=1:rg[i]
                if length(intersect(monomials(Tg[i][p,q]),U))!=0
                    C[p,q]=1
                end
            end
        end
        C[diagind(C)] .= 0
        push!(Cg,C)
    end
        
    Ch=[]
    for i=1:l
        C=zeros(rh[i],rh[i])

        for p=1:rh[i]
            for q=1:rh[i]
                if length(intersect(monomials(Th[i][p,q]),U))!=0
                    C[p,q]=1
                end
            end
        end
        C[diagind(C)] .= 0
        push!(Ch,C)
    end
    
    #maximal cliques

    grap = SimpleGraph(Cf)
    
    ind_block_f=connected_components(grap)
    
    length_ind_block_f=length(ind_block_f)
    
    for j=1:length_ind_block_f
        Uf=[Uf;vec(v[ind_block_f[j]]*v[ind_block_f[j]]')]  
    end
    Uf=unique(Uf)
    
    for i=1:m
        
        grap = SimpleGraph(Cg[i])

        ind_block_g[i]=connected_components(grap)

        length_ind_block_g[i]=length(ind_block_g[i])

        for j=1:length_ind_block_g[i]
            mono=monomials(g[i])
            Ug[i]=vec(mono[1]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]')
            for p=2:length(mono)
                Ug[i]=union(Ug[i],vec(mono[p]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]'))
            end
        end
    end
    
    

    
    for i=1:l
        
        grap = SimpleGraph(Ch[i])

        ind_block_h[i]=connected_components(grap)

        length_ind_block_h[i]=length(ind_block_h[i])

        for j=1:length_ind_block_h[i]
            mono=monomials(h[i])
            Uh[i]=vec(mono[1]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]')
            for p=2:length(mono)
                Uh[i]=union(Uh[i],vec(mono[p]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]'))
            end
        end
    end    
  
end

elapsed = time() - start
println("elapsed time = ",elapsed)

In [ ]:
ind_block_f

In [ ]:
ind_block_g

In [ ]:
ind_block_h

In [ ]:
model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))

wSOS=0

G=[]
for i=1:length_ind_block_f
    push!(G, @variable(model, [1:length(ind_block_f[i]), 1:length(ind_block_f[i])],PSD)) 
end

for i=1:length_ind_block_f
    wSOS = wSOS + v[ind_block_f[i]]'*G[i]*v[ind_block_f[i]]   
end


Gg=[]
for j=1:m
    push!(Gg,[])
    for i=1:length_ind_block_g[j]
        push!(Gg[j], @variable(model, [1:length(ind_block_g[j][i]), 1:length(ind_block_g[j][i])],PSD))
    end
end

for j=1:m
    for i=1:length_ind_block_g[j]
        wSOS = wSOS + g[j]*v[ind_block_g[j][i]]'*Gg[j][i]*v[ind_block_g[j][i]]   
    end
end


Gh=[]
length_Gh=[]
vh=[]
for j=1:l
    push!(Gh,[])
    push!(length_Gh,[])
    push!(vh,[])
    for i=1:length_ind_block_h[j]
        push!(vh[j], unique(vec(v[ind_block_h[j][i]]*v[ind_block_h[j][i]]')))
        push!(length_Gh[j], length(vh[j][i]))
        push!(Gh[j], @variable(model, [1:length_Gh[j][i]]))
    end
end

for j=1:l
    for i=1:length_ind_block_h[j]
        wSOS = wSOS + h[j]*vh[j][i]'*Gh[j][i]   
    end
end



@variable(model, lambda)

@constraint(model, coefficients(f-lambda - wSOS).== 0)

@objective(model, Max, lambda)

optimize!(model)

omega0 = value(lambda)

println("termination status = ", termination_status(model))

println("omega",0," = ", omega0)

elapsed = time() - start
println("elapsed time = ",elapsed)

println("---------------------------------------------------------------------------------------")

G0=zeros(rf,rf)
for i=1:length_ind_block_f
    G0[ind_block_f[i],ind_block_f[i]]=value.(G[i]) 
end

using RowEchelon

r=rf-rank(G0, TOL)

println("rank of moment matrix = ", r)


# extraction of Henrion and Lasserre
F = eigen(G0)
V = F.vectors

Ix=sortperm(F.values)

V=V[:,Ix[1:r]]
V=Matrix(V')
V= rref_with_pivots!(V,tau);
U=V[1]

U=Matrix(U')
# Figure out multiplying matrices using YALMIP code
w=v[V[2]];
N=zeros(length(V[2]),r,n)
for i in 1:n
    xw=x[i]*w
    kk=indexin(xw,v)
    N[:,:,i]=U[kk,:]
end



# Create random convex combination
rands = rand(n,1);rands = rands/sum(rands);
M = zeros(length(V[2]),r);
for i in 1:n
    M=M+rands[i]*N[:,:,i];
end

F= schur(M);
L=F.Z
# Extract solution
for i in 1:r
    solution=[]
    for j = 1:n
        solution=[solution;L[:,i]'*N[:,:,j]*L[:,i]];
    end

    println("solution = ",solution)
    #check the feasibility of solution 
    
    
    for i in 1:m
        println("check inequality ",i," = ",polynomial(g[i])(x => solution))         
    end

    for i in 1:l
        println("check equality ",i," = ",polynomial(h[i])(x => solution))
    end
    elapsed = time() - start
    println("elapsed time = ",elapsed)
    println("---------------------------------------------------------------------------------------")
end



println("=======================================================================================")



In [ ]:
#equalities polynomial
if l==0
    h=[omega0-f]
else
    h=[h;omega0-f]
end
l=length(h)

#inequalities polynomial
g=g[1:end-1] ; m=length(g)

for t=1:n

    println("Determine omega",t,":")

    println("---------------------------------------------------------------------------------------")

    

    if t>1
        #equalities polynomial
        h = [h;omega[t-1]-f] ; l=length(h)
    end        

    f=[x-a[:,t]]'*[x-a[:,t]]

    # Degree of inequalities polynomials
    dg = []
    for i = 1:m
        dg = [dg; ceil(Int,DynamicPolynomials.degree(leadingmonomial(g[i]))/2)]
    end

    dh = []
    for i = 1:l
        dh = [dh; ceil(Int,DynamicPolynomials.degree(leadingmonomial(h[i]))/2)]
    end
    
    
    rf=binomial(k+d+n,n)

    rg=[];
    for i = 1:m
        rg=[rg;binomial(k+d-dg[i]+n,n)]
    end

    rh=[];
    for i = 1:l
        rh=[rh;binomial(k+d-dh[i]+n,n)]
    end


    Uf=union(v.^2,monomials(f))

    Ug=[]
    for i = 1:m
        Uf=union(Uf,monomials(g[i]))
        push!(Ug,[])
    end

    Uh=[]
    for i = 1:l
        Uf=union(Uf,monomials(h[i]))
        push!(Uh,[])
    end


    Tf=v*v'
    Tg=[]
    for i=1:m
        push!(Tg,g[i].*v[1:rg[i]]*v[1:rg[i]]')
    end

    Th=[]
    for i=1:l
        push!(Th,h[i].*v[1:rh[i]]*v[1:rh[i]]')
    end



    length_ind_block_f=[]
    ind_block_f=[]

    length_ind_block_g=[]
    ind_block_g=[]

    for i=1:m
        push!(length_ind_block_g,[])
        push!(ind_block_g,[])
    end

    length_ind_block_h=[]
    ind_block_h=[]

    for i=1:l
        push!(length_ind_block_h,[])
        push!(ind_block_h,[])
    end


    for t=0:k_sparse-1
        U=Uf
        for i=1:m
            if length(Ug[i])!=0
                U=union(U,Ug[i])
            end
        end
        for i=1:l
            if length(Uh[i])!=0
                U=union(U,Uh[i])
            end
        end

        Cf=zeros(rf,rf)

        Cf[findall(p -> p in U, Tf)].=1

        Cf[diagind(Cf)] .= 0

        Cg=[]
        for i=1:m
            C=zeros(rg[i],rg[i])

            for p=1:rg[i]
                for q=1:rg[i]
                    if length(intersect(monomials(Tg[i][p,q]),U))!=0
                        C[p,q]=1
                    end
                end
            end
            C[diagind(C)] .= 0
            push!(Cg,C)
        end

        Ch=[]
        for i=1:l
            C=zeros(rh[i],rh[i])

            for p=1:rh[i]
                for q=1:rh[i]
                    if length(intersect(monomials(Th[i][p,q]),U))!=0
                        C[p,q]=1
                    end
                end
            end
            C[diagind(C)] .= 0
            push!(Ch,C)
        end

        #maximal cliques

        grap = SimpleGraph(Cf)

        ind_block_f=connected_components(grap)

        length_ind_block_f=length(ind_block_f)

        for j=1:length_ind_block_f
            Uf=[Uf;vec(v[ind_block_f[j]]*v[ind_block_f[j]]')]  
        end
        Uf=unique(Uf)

        for i=1:m

            grap = SimpleGraph(Cg[i])

            ind_block_g[i]=connected_components(grap)

            length_ind_block_g[i]=length(ind_block_g[i])

            for j=1:length_ind_block_g[i]
                mono=monomials(g[i])
                Ug[i]=vec(mono[1]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]')
                for p=2:length(mono)
                    Ug[i]=union(Ug[i],vec(mono[p]*v[ind_block_g[i][j]]*v[ind_block_g[i][j]]'))
                end
            end
        end




        for i=1:l

            grap = SimpleGraph(Ch[i])

            ind_block_h[i]=connected_components(grap)

            length_ind_block_h[i]=length(ind_block_h[i])

            for j=1:length_ind_block_h[i]
                mono=monomials(h[i])
                Uh[i]=vec(mono[1]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]')
                for p=2:length(mono)
                    Uh[i]=union(Uh[i],vec(mono[p]*v[ind_block_h[i][j]]*v[ind_block_h[i][j]]'))
                end
            end
        end    

    end

    elapsed = time() - start
    println("elapsed time = ",elapsed)



    model = Model(with_optimizer(Mosek.Optimizer, QUIET=true))

    wSOS=0

    G=[]
    for i=1:length_ind_block_f
        push!(G, @variable(model, [1:length(ind_block_f[i]), 1:length(ind_block_f[i])],PSD)) 
    end

    for i=1:length_ind_block_f
        wSOS = wSOS + v[ind_block_f[i]]'*G[i]*v[ind_block_f[i]]   
    end


    Gg=[]
    for j=1:m
        push!(Gg,[])
        for i=1:length_ind_block_g[j]
            push!(Gg[j], @variable(model, [1:length(ind_block_g[j][i]), 1:length(ind_block_g[j][i])],PSD))
        end
    end

    for j=1:m
        for i=1:length_ind_block_g[j]
            wSOS = wSOS + g[j]*v[ind_block_g[j][i]]'*Gg[j][i]*v[ind_block_g[j][i]]   
        end
    end


    Gh=[]
    length_Gh=[]
    vh=[]
    for j=1:l
        push!(Gh,[])
        push!(length_Gh,[])
        push!(vh,[])
        for i=1:length_ind_block_h[j]
            push!(vh[j], unique(vec(v[ind_block_h[j][i]]*v[ind_block_h[j][i]]')))
            push!(length_Gh[j], length(vh[j][i]))
            push!(Gh[j], @variable(model, [1:length_Gh[j][i]]))
        end
    end

    for j=1:l
        for i=1:length_ind_block_h[j]
            wSOS = wSOS + h[j]*vh[j][i]'*Gh[j][i]   
        end
    end



    @variable(model, lambda)

    @constraint(model, coefficients(f-lambda - wSOS).== 0)

    @objective(model, Max, lambda)

    optimize!(model)

    omega[t] = value(lambda)

    println("termination status = ", termination_status(model))

    println("omega",t," = ", omega[t])

    elapsed = time() - start
    println("elapsed time = ",elapsed)

    println("---------------------------------------------------------------------------------------")

    G0=zeros(rf,rf)
    for i=1:length_ind_block_f
        G0[ind_block_f[i],ind_block_f[i]]=value.(G[i]) 
    end


    r=rf-rank(G0, TOL)

    println("rank of moment matrix = ", r)


    # extraction of Henrion and Lasserre
    F = eigen(G0)
    V = F.vectors

    Ix=sortperm(F.values)

    V=V[:,Ix[1:r]]
    V=Matrix(V')
    V= rref_with_pivots!(V,tau);
    U=V[1]

    U=Matrix(U')
    # Figure out multiplying matrices using YALMIP code
    w=v[V[2]];
    N=zeros(length(V[2]),r,n)
    for i in 1:n
        xw=x[i]*w
        kk=indexin(xw,v)
        N[:,:,i]=U[kk,:]
    end



    # Create random convex combination
    rands = rand(n,1);rands = rands/sum(rands);
    M = zeros(length(V[2]),r);
    for i in 1:n
        M=M+rands[i]*N[:,:,i];
    end

    F= schur(M);
    L=F.Z
    # Extract solution
    for i in 1:r
        solution=[]
        for j = 1:n
            solution=[solution;L[:,i]'*N[:,:,j]*L[:,i]];
        end

        println("solution = ",solution)
        #check the feasibility of solution 


        for i in 1:m
            println("check inequality ",i," = ",polynomial(g[i])(x => solution))         
        end

        for i in 1:l
            println("check equality ",i," = ",polynomial(h[i])(x => solution))
        end
        elapsed = time() - start
        println("elapsed time = ",elapsed)
        println("---------------------------------------------------------------------------------------")
    end



    println("=======================================================================================")

end